<a href="https://colab.research.google.com/github/nam-hoang-le/Fine-tuning-LLMs-for-conversation/blob/le-hoang-nam/VinaLLaMA_for_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install and import necessary libaries:

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops
!pip install -q -U googletrans==3.1.0a0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 901.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2

In [ ]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from googletrans import Translator
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## 2. Sign in to huggingface

In [ ]:
notebook_login()

## 3. Load pretrained LLM

In [ ]:
# khởi tạo mô hình
MODEL_NAME = "vilm/vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
    )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 39976960 || all params: 3657576448 || trainables%: 1.092990415056391


## 4. Test pretrained model performance (make prediction)

In [ ]:
# khởi tạo thông số
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
# khai báo prompt
prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
chào bạn nhé !
<|im_end|>
<|im_start|>assistant
""".strip()

In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
chào bạn nhé !
 
<|im_start|> assistant
Chào bạn! Làm thế nào tôi có thể giúp bạn hôm nay? Nếu bạn có bất kỳ câu hỏi nào hoặc cần sự trợ giúp, hãy thoải mái hỏi.
CPU times: user 8.86 s, sys: 928 ms, total: 9.79 s
Wall time: 14 s


## 5. Fine-tuning LLM

### 5.1. Prepare dataset

In [ ]:
data = load_dataset('alespalla/chatbot_instruction_prompts')

Generating train split:   0%|          | 0/258042 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/64511 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt'],
        num_rows: 258042
    })
    test: Dataset({
        features: ['response', 'prompt'],
        num_rows: 64511
    })
})

In [ ]:
type(data)

datasets.dataset_dict.DatasetDict

In [ ]:
data["train"][15]

{'response': 'Mitten.', 'prompt': 'Produce a random noun. '}

In [ ]:
# Xây dựng bộ dữ liệu fine-tuning

In [ ]:
# Xây dựng hàm tạo prompt
def generate_prompt(data_point):
    translator = Translator()
    vn_prompt = translator.translate(data_point['prompt'], src='en', dest='vi').text
    vn_response = translator.translate(data_point['response'], src='en', dest='vi').text

    return f"""
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
{vn_prompt}
<|im_end|>
<|im_start|>assistant
{vn_response}
""".strip()



In [1]:
# Xây dựng hàm tạo tokenization

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)

    return tokenized_full_prompt

In [ ]:
# Áp dụng okenization vào bộ dữ liệu
data = data['train'].shard(num_shards=50, index=0).filter(lambda sample: sample['response'] != '' and
                                                          sample['prompt'] != '').shuffle().map(generate_and_tokenize_prompt)

Filter:   0%|          | 0/5161 [00:00<?, ? examples/s]

Map:   0%|          | 0/5161 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### 5.2. Training

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

from torch.utils.data import Subset

# Assuming `data` is a torch.utils.data.Dataset
indices = range(len(data) // 4)  # Indices for a quarter of the dataset
smaller_data = Subset(data, indices)

trainer = transformers.Trainer(
    model=model,
    train_dataset= smaller_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.665500
2,3.935300
3,3.689800
4,4.126600
5,3.719600
6,2.346900
7,3.312500
8,2.841900
9,3.080900
10,2.863800


Step,Training Loss
1,1.078900
2,1.090900
3,1.046400
4,1.101800
5,0.904200
6,0.925500
7,0.926500
8,0.991500
9,1.014200
10,1.167000


TrainOutput(global_step=322, training_loss=1.1156082828963025, metrics={'train_runtime': 2203.0868, 'train_samples_per_second': 0.586, 'train_steps_per_second': 0.146, 'total_flos': 7264476244721664.0, 'train_loss': 1.1156082828963025, 'epoch': 1.0})

### 5.3. Test prediction

In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Mô tả về thành phố Đà Lạt
<|im_end|>
<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Mô tả về thành phố Đà Lạt
 
<|im_start|> assistant
Đà Lạt là một thành phố đẹp như tranh vẽ nằm ở vùng cao nguyên Nam Bộ của Việt Nam. Được biết đến như là “Thủ đô hoa của Việt Nam”, thành phố này nổi tiếng với những khu vườn tươi tốt, những ngôi nhà đẹp và những món ăn ngon. Thành phố được bao quanh bởi những ngọn núi và có khí hậu mát mẻ, mang đến cho du khách cơ hội khám phá những con đường mòn đi bộ đường dài tuyệt đẹp và tận hưởng những hoạt động ngoài trời như đi bè, câu cá và cưỡi ngựa. Đà Lạt cũng nổi tiếng với những lễ hội văn hóa sôi động, các lễ hội ẩm thực và các buổi biểu diễn âm nhạc và sân khấu. Thành phố này là nơi có nhiều nhà hàng, quán cà phê và các điểm mua sắm, mang đến cho du khách cơ hội trải nghiệm cuộc sống về đêm sôi động. Đà Lạt là một thành phố xinh đẹp và thanh bình, là nơi lý tưởng cho những người tìm kiếm một kỳ nghỉ yên
CPU times: user 2min 18

## 6. Save model to huggingface

In [ ]:
model.save_pretrained("trained-model")

## 7. Inference

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
kể cho tôi một câu chuyện hài hước
<|im_end|>
<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
kể cho tôi một câu chuyện hài hước
 
<|im_start|> assistant
Một ngày nọ, một người đàn ông bước vào một cửa hàng tạp hóa và nói với người bán hàng, "Tôi muốn mua một quả táo." Người bán hàng hỏi, "Anh muốn mua loại táo nào?" Người đàn ông trả lời, "Tôi muốn một quả táo có màu đỏ." Người bán hàng hỏi, "Anh có muốn mua một quả táo có màu đỏ không?" Người đàn ông trả lời, "Không, tôi chỉ muốn một quả táo có màu đỏ." Người bán hàng trả lời, "Được rồi, tôi sẽ mang cho anh một quả táo có màu đỏ." Người đàn ông trả lời, "Tôi chỉ muốn một quả táo có màu đỏ, không phải màu đỏ." Người bán hàng trả lời, "Được rồi, tôi sẽ mang cho anh một quả táo có màu đỏ." Người đàn ông trả lời, "Tôi chỉ muốn một quả táo có màu đỏ, không phải màu đỏ." Người bán hàng trả lời, "Được rồi, tôi sẽ mang cho anh
CPU times: user 36.7 s, sys: 281 ms, total: 37 s
Wall time: 1min 2s


In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = """
<|im_start|>system
Bạn là một nhà tiên tri thần sầu, hãy trả lời câu hỏi của ta
<|im_end|>
<|im_start|>user
Bơ thúi là biệt danh của người yêu tôi, hôm nay Bơ đi thi Tiếng Anh. Vậy nói cho tui Bơ thúi đi thi được mấy điểm ?
<|im_end|>
<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<|im_start|> system
Bạn là một nhà tiên tri thần sầu, hãy trả lời câu hỏi của ta
 
<|im_start|> user
Bơ thúi là biệt danh của người yêu tôi, hôm nay Bơ đi thi Tiếng Anh. Vậy nói cho tui Bơ thúi đi thi được mấy điểm ?
 
<|im_start|> assistant
Tôi không chắc chắn về ý nghĩa của từ này. Bạn có thể giải thích thêm về nó được không? Bạn có muốn nói rằng bạn không thích người yêu của mình? Bạn có thể giải thích thêm về điều đó được không? Bạn có muốn nói rằng bạn đang cố gắng làm cho người yêu của mình thất bại trong kỳ thi của họ không? Bạn có thể giải thích thêm về điều đó được không? Bạn có muốn nói rằng bạn đang cố gắng làm cho người yêu của mình thất bại trong kỳ thi của họ không? Bạn có thể giải thích thêm về điều đó được không? Bạn có muốn nói rằng bạn đang cố gắng làm cho người yêu của mình thất bại trong kỳ thi của họ không? Bạn có thể giải thích thêm về điều đó được không? Bạn có muốn nói rằng bạn đang cố gắng làm cho người yêu của mình thất bại trong kỳ thi của họ không? Bạn có th